In [3]:
import tensorflow as tf
import tensorlayer as tl

tl.layers.set_keep['_layers_name_list']=[]
tf.reset_default_graph()
sess = tf.InteractiveSession()

# 准备数据
X_train, y_train, X_val, y_val, X_test, y_test = \
                                tl.files.load_mnist_dataset(shape=(-1,784))

# 定义 placeholder
x = tf.placeholder(tf.float32, shape=[None, 784], name='x')
y_ = tf.placeholder(tf.int64, shape=[None, ], name='y_')

# 定义模型
network = tl.layers.InputLayer(x, name='input_layer')
network = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')
network = tl.layers.DenseLayer(network, n_units=800,
                                act = tf.nn.relu, name='relu1')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')
network = tl.layers.DenseLayer(network, n_units=800,
                                act = tf.nn.relu, name='relu2')
network = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')
network = tl.layers.DenseLayer(network, n_units=10,
                                act = tf.identity,
                                name='output_layer')
# 定义损失函数和衡量指标
# tl.cost.cross_entropy 在内部使用 tf.nn.sparse_softmax_cross_entropy_with_logits() 实现 softmax
y = network.outputs
cost = tl.cost.cross_entropy(y, y_)
correct_prediction = tf.equal(tf.argmax(y, 1), y_)
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
y_op = tf.argmax(tf.nn.softmax(y), 1)

# 定义 optimizer
train_params = network.all_params
train_op = tf.train.AdamOptimizer(learning_rate=0.0001, beta1=0.9, beta2=0.999,
                            epsilon=1e-08, use_locking=False).minimize(cost, var_list=train_params)

# 初始化所有参数
sess.run(tf.initialize_all_variables())

# 列出模型信息
network.print_params()
network.print_layers()

# 训练模型
tl.utils.fit(sess, network, train_op, cost, X_train, y_train, x, y_,
            acc=acc, batch_size=500, n_epoch=500, print_freq=5,
            X_val=X_val, y_val=y_val, eval_train=False)

# 评估模型
tl.utils.test(sess, network, acc, X_test, y_test, x, y_, batch_size=None, cost=cost)

# 把模型保存成 .npz 文件
tl.files.save_npz(network.all_params , name='model.npz')
sess.close()

  tensorlayer:Instantiate InputLayer  input_layer: (?, 784)
  tensorlayer:Instantiate DropoutLayer drop1: keep: 0.800000
  tensorlayer:Instantiate DenseLayer  relu1: 800, relu
  tensorlayer:Instantiate DropoutLayer drop2: keep: 0.500000
  tensorlayer:Instantiate DenseLayer  relu2: 800, relu
  tensorlayer:Instantiate DropoutLayer drop3: keep: 0.500000
  tensorlayer:Instantiate DenseLayer  output_layer: 10, identity
  param   0: (784, 800)      (mean: 0.000123850622913 , median: 0.00024971953826  , std: 0.0880593210459   )   relu1/W:0
  param   1: (800,)          (mean: 0.0               , median: 0.0               , std: 0.0               )   relu1/b:0
  param   2: (800, 800)      (mean: -7.63565913076e-05, median: -9.57919546636e-05, std: 0.088065892458    )   relu2/W:0
  param   3: (800,)          (mean: 0.0               , median: 0.0               , std: 0.0               )   relu2/b:0
  param   4: (800, 10)       (mean: 0.00121473788749  , median: 0.00167763885111  , std: 0.0876733

Exception AssertionError: AssertionError("Nesting violated for default stack of <type 'weakref'> objects",) in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7fd0b0455190>> ignored


KeyboardInterrupt: 

In [3]:
network.print_params()

  param   0: (784, 800)      (mean: -0.00708959717304 , median: -0.00657796068117 , std: 0.0905798822641   )   relu1/W:0
  param   1: (800,)          (mean: -0.0162756498903  , median: -0.0165478996933  , std: 0.0286038387567   )   relu1/b:0
  param   2: (800, 800)      (mean: -0.0075948019512  , median: -0.00770991016179 , std: 0.0889174118638   )   relu2/W:0
  param   3: (800,)          (mean: -0.00714833196253 , median: -0.00884392485023 , std: 0.0469182692468   )   relu2/b:0
  param   4: (800, 10)       (mean: -0.0036618472077  , median: 0.000972303911112 , std: 0.0824688673019   )   output_layer/W:0
  param   5: (10,)           (mean: -0.00623427052051 , median: -0.0236124619842  , std: 0.0789439752698   )   output_layer/b:0
  num of params: 1276810


In [7]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np
# Helper functions for TF Graph visualization
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, 'utf-8')
    return strip_def
  

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))
    

In [8]:
show_graph(tf.get_default_graph())

In [27]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [28]:
globals()

{'In': ['',
  u"import tensorflow as tf\nimport tensorlayer as tl\n\nsess = tf.InteractiveSession()\n\n# \u51c6\u5907\u6570\u636e\nX_train, y_train, X_val, y_val, X_test, y_test =                                 tl.files.load_mnist_dataset(shape=(-1,784))\n\n# \u5b9a\u4e49 placeholder\nx = tf.placeholder(tf.float32, shape=[None, 784], name='x')\ny_ = tf.placeholder(tf.int64, shape=[None, ], name='y_')\n\n# \u5b9a\u4e49\u6a21\u578b\nnetwork = tl.layers.InputLayer(x, name='input_layer')\nnetwork = tl.layers.DropoutLayer(network, keep=0.8, name='drop1')\nnetwork = tl.layers.DenseLayer(network, n_units=800,\n                                act = tf.nn.relu, name='relu1')\nnetwork = tl.layers.DropoutLayer(network, keep=0.5, name='drop2')\nnetwork = tl.layers.DenseLayer(network, n_units=800,\n                                act = tf.nn.relu, name='relu2')\nnetwork = tl.layers.DropoutLayer(network, keep=0.5, name='drop3')\nnetwork = tl.layers.DenseLayer(network, n_units=10,\n                 